In [1]:
'''
Zomato Restaurants Data

- Processing JSON Files returned from the Zomato /search API endpoint
https://developers.zomato.com/documentation#!/restaurant/search

StackOverflow:
https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s

'''

'\nZomato Restaurants Data\n\n- Processing JSON Files returned from the Zomato /search API endpoint\nhttps://developers.zomato.com/documentation#!/restaurant/search\n\nStackOverflow:\nhttps://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s\n\n'

In [2]:
# Import libraries
from datetime import datetime
from glob import glob
from pandas import json_normalize

import json
import numpy as np
import os
import pandas as pd 
import zipfile

In [3]:
file = r"/Users/kevingrimm_/Desktop/ETL/GitHub Projects/zomato-restaurants-data.zip"

In [4]:
# extract zip files
with zipfile.ZipFile(file) as zip_file:
    zip_file.extractall()

In [5]:
# get list of files in current directory; confirm that zip was unzipped
files = [f for f in os.listdir() if os.path.isfile(f)]
files

['Restaurant_Data_08-04-2020.xlsx',
 'file5.json',
 '.DS_Store',
 'file4.json',
 'Untitled.ipynb',
 'file3.json',
 'Zomato - Processing JSON Files.ipynb',
 'zomato.csv',
 'file2.json',
 'file1.json',
 'zomato-restaurants-data.zip',
 'Country-Code.xlsx']

In [6]:
'''
Per Kaggle, the cleaned data to be used for analysis is stored in 'zomato.csv'. 

However, for the purpose of this exercise, we are interested in the raw .json files.
The aim is to produce an ETL that achieves the following:

(1) Calls various API endpoints to extract raw data (and metadata) from sources
(2) Processes the raw files to recreate the collected data
(3) Create visualizations for cleaned data to reveal insights

This Jupyter notebook will initially only cover the data extracting, exploratory analysis, and data processing.
Later, modules will be produced to enable the following:
(1) Convert entire process into a DAG that runs on Apache Airflow
(2) Store raw data in S3
(3) Produce a Flask app that displays metrics for various dimensions (e.g., restaurant, city, country)
'''

"\nPer Kaggle, the cleaned data to be used for analysis is stored in 'zomato.csv'. \n\nHowever, for the purpose of this exercise, we are interested in the raw .json files.\nThe aim is to produce an ETL that achieves the following:\n\n(1) Calls various API endpoints to extract raw data (and metadata) from sources\n(2) Processes the raw files to recreate the collected data\n(3) Create visualizations for cleaned data to reveal insights\n\nThis Jupyter notebook will initially only cover the data extracting, exploratory analysis, and data processing.\nLater, modules will be produced to enable the following:\n(1) Convert entire process into a DAG that runs on Apache Airflow\n(2) Store raw data in S3\n(3) Produce a Flask app that displays metrics for various dimensions (e.g., restaurant, city, country)\n"

In [7]:
# In order to process each JSON file, we need to better understand the file contents.

# Pulling all JSON files into the currenty directory
json_files = glob("*.json")

# Confirming that JSON files are in a list
json_files

['file5.json', 'file4.json', 'file3.json', 'file2.json', 'file1.json']

In [8]:
# Assigning a JSON file to a variable for further exploration
first_json = json_files[0]

# Opening the file, loading into a variable
with open(first_json) as f:
    df = json.load(f)

In [9]:
'''
300 items are contained within var `df`
Now, we are going to review individual results to identify key/value patterns
'''
len(df)

300

In [10]:
'''
The first object has four keys, yet no values are stored in the dict
- Moving on to object #2 to see if results are different
'''

r1 = df[0]
print(r1.keys())
r1

dict_keys(['results_found', 'restaurants', 'results_shown', 'results_start'])


{'results_found': 0,
 'restaurants': [],
 'results_shown': 0,
 'results_start': '1'}

In [11]:
'''
Object #2 returned results, and the data we are interested in is stored as a list value for `restaurants`
'''
r2 = df[1]
print(r2.keys())
print('Results found: ', r2['results_found'])
print('Total restaurants: ', len(r2['restaurants']))
print('Results shown: ', r2['results_shown'])
print('Results start: ', r2['results_start'])

dict_keys(['results_found', 'restaurants', 'results_shown', 'results_start'])
Results found:  6835
Total restaurants:  20
Results shown:  20
Results start:  1


In [26]:
'''
Assigning a var to the first item; checking it's data type
'''

rest1 = r2['restaurants'][0]
type(rest1)

dict

In [27]:
# Checking keys now that we know it is a dict
rest1.keys()

dict_keys(['restaurant'])

In [28]:
# Setting the var to the contents of it's lone key; checking data type
rest1 = rest1['restaurant']

type(rest1)

dict

In [30]:
'''
This object reveals 23 keys
- At least one object within the outer object can be filtered out ({'...'} dict)
'''
print(len(rest1.keys()))
rest1.keys()

23


dict_keys(['has_online_delivery', 'photos_url', 'url', 'price_range', 'apikey', 'user_rating', 'R', 'name', 'has_table_booking', 'is_delivering_now', 'deeplink', 'menu_url', 'average_cost_for_two', 'switch_to_order_menu', 'offers', 'cuisines', 'location', 'featured_image', 'currency', 'id', 'thumb', 'establishment_types', 'events_url'])

In [31]:
'''
Reviewing the object at a high level
- Following kernels dig into each key/value pair to assess their relevance to analytics
(referenced API documentation for definitions)
'''
rest1

{'has_online_delivery': 0,
 'photos_url': 'https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park/photos?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1#tabtop',
 'url': 'https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
 'price_range': 3,
 'apikey': 'b90e6a8c738410315a20c449fe2eb1b1',
 'user_rating': {'rating_text': 'Very Good',
  'rating_color': '5BA829',
  'votes': '568',
  'aggregate_rating': '4.2'},
 'R': {'res_id': 17057797},
 'name': 'Bosphorous Turkish Cuisine',
 'has_table_booking': 0,
 'is_delivering_now': 0,
 'deeplink': 'zomato://restaurant/17057797',
 'menu_url': 'https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park/menu?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1&openSwipeBox=menu&showMinimal=1#tabtop',
 'average_cost_for_two': 40,
 'switch_to_order_menu': 0,
 'offers': [],
 'cuisines': 'Mediterranean, Turkish',
 'location': {'latitude

In [32]:
# Likely candiate for Yes/No values - need to review other outputs to confirm
rest1['has_online_delivery']

0

In [33]:
# Not particularly helpful information for analytics
rest1['photos_url']

'https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park/photos?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1#tabtop'

In [34]:
# Not particularly helpful for analytics
rest1['url']

'https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1'

In [35]:
# Price range is likely on a 1-5 or something similar - relevant dimension
rest1['price_range']

3

In [36]:
# Not relevant for analytics - API key for the original user
rest1['apikey']

'b90e6a8c738410315a20c449fe2eb1b1'

In [37]:
# These fields can be normalized as individual columns
rest1['user_rating']

{'rating_text': 'Very Good',
 'rating_color': '5BA829',
 'votes': '568',
 'aggregate_rating': '4.2'}

In [38]:
# restaurant ID is nested - relevant metric (for analytics/mapping)
rest1['R']

{'res_id': 17057797}

In [39]:
# restaurant name - relevant dimension
rest1['name']

'Bosphorous Turkish Cuisine'

In [40]:
# Whether the restaurant has table reservation enabled - relevant
rest1['has_table_booking']

0

In [41]:
# Valid only if has_online_delivery =1; whether it is accepting online orders right now - relevant
rest1['is_delivering_now']

0

In [42]:
# short URL of the restaurant page
# Not relevant for analytics
rest1['deeplink']

'zomato://restaurant/17057797'

In [43]:
# URL of the menu page
# Not relevant for analytics
rest1['menu_url']

'https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park/menu?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1&openSwipeBox=menu&showMinimal=1#tabtop'

In [44]:
# Average price of a meal for two people - relevant
rest1['average_cost_for_two']

40

In [45]:
# not referenced in API documentation
# Possible candidate for Boolean dtype - keep for now
rest1['switch_to_order_menu']

0

In [46]:
# not referenced in API documentation
# emtpy list - likely will discard
rest1['offers']

[]

In [47]:
# list of cuisines served at restaurant - relevant
rest1['cuisines']

'Mediterranean, Turkish'

In [48]:
# location data can be normalized into individual columns - relevant for analytics/mapping
rest1['location']

{'latitude': '28.5976271000',
 'address': '108 S Park Ave, Winter Park, FL 32789',
 'city': 'Orlando',
 'country_id': 216,
 'locality_verbose': 'Winter Park, Orlando',
 'city_id': 601,
 'zipcode': '32789',
 'longitude': '-81.3508344000',
 'locality': 'Winter Park'}

In [49]:
# URL of high resolution header image
# Not particularly relevant for analytics
rest1['featured_image']

'https://b.zmtcdn.com/data/pictures/7/17057797/6504549ba9a80220f1d891b4fc0f0e72_featured_v2.jpg'

In [50]:
# local currency symbol; to be used with price
# Can add a currency mapping for USD conversions (would need data source)
rest1['currency']

'$'

In [51]:
# ID of the restaurant - relevant
# Matches value in rest['R']['rest_id']
# Only need one of two fields, so keep one and drop the other
rest1['id']

'17057797'

In [52]:
# URL of the low resolution header
# Not relevant for anlytics 
rest1['thumb']

'https://b.zmtcdn.com/data/pictures/7/17057797/6504549ba9a80220f1d891b4fc0f0e72_featured_v2.jpg'

In [53]:
# Need to review other results to determine relevance (i.e. are empty lists always returned)
# Can possibly use the /establishments endpoint to add an establishments mapping
rest1['establishment_types']

[]

In [54]:
# URL of the restaurants events page
# Not particularly relevant for analytics
rest1['events_url']

'https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park/events#tabtop?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1'

In [55]:
'''
NOTES:
- After an initial review of the data, we can split the keys into two categories
- Assuming that the listed fields are normalized if they are stored in a nested dict.
- `offers` is not referenced in documentation and returned an empty list, so discarding for now
- `switch_to_order_menu` is not referenced in documentation but has results, so keeping for now
- `establishment_type` can potentially be pulled from the /establishments endpoint (still need results)  
- some location fields (e.g., `city_id`, `country_id`, `zipcode`, can be saved in lookup tables)

KEEP:
- establishment_type
- id
- name
- country_id
- city
- city_id
- zipcode
- address
- locality
- locality_verbose
- longitude
- latitude
- cuisines
- average_cost_for_two
- currency
- has_table_booking
- has_online_delivery
- is_delivering_now
- switch_to_order_menu 
- price_range
- aggregate_rating
- rating_color
- rating_text
- votes

DISCARDD:
- events_url
- thumb
- featured_image
- menu_url
- deeplink
- R
- apikey
- url
- photos_url
- offers (No API documentation)
'''
print()

In [56]:
'''
Using pandas.json_normalize to normalize the results. Before processing the results, need to codify the 
process for converting all results. 

- Test process for a single result
- Convert into function
- Test function with a handle of results
- If OK, apply to all results in JSON
'''
data = json_normalize(rest1)
data.columns

Index(['has_online_delivery', 'photos_url', 'url', 'price_range', 'apikey',
       'name', 'has_table_booking', 'is_delivering_now', 'deeplink',
       'menu_url', 'average_cost_for_two', 'switch_to_order_menu', 'offers',
       'cuisines', 'featured_image', 'currency', 'id', 'thumb',
       'establishment_types', 'events_url', 'user_rating.rating_text',
       'user_rating.rating_color', 'user_rating.votes',
       'user_rating.aggregate_rating', 'R.res_id', 'location.latitude',
       'location.address', 'location.city', 'location.country_id',
       'location.locality_verbose', 'location.city_id', 'location.zipcode',
       'location.longitude', 'location.locality'],
      dtype='object')

In [57]:
# columns to keep
keep = ['has_online_delivery', 'price_range', 'name', 'has_table_booking', 'is_delivering_now', 
       'average_cost_for_two', 'switch_to_order_menu', 'cuisines', 'currency', 'id', 'establishment_types',
       'user_rating.rating_text', 'user_rating.rating_color', 'user_rating.votes', 'user_rating.aggregate_rating',
       'location.latitude', 'location.address', 'location.city', 'location.city_id', 'location.country_id', 
       'location.locality_verbose', 'location.zipcode', 'location.locality', 'location.longitude']
len(keep)

24

In [58]:
# filter data to desired columns
data = data[keep]
# check dtypes
data.dtypes

has_online_delivery              int64
price_range                      int64
name                            object
has_table_booking                int64
is_delivering_now                int64
average_cost_for_two             int64
switch_to_order_menu             int64
cuisines                        object
currency                        object
id                              object
establishment_types             object
user_rating.rating_text         object
user_rating.rating_color        object
user_rating.votes               object
user_rating.aggregate_rating    object
location.latitude               object
location.address                object
location.city                   object
location.city_id                 int64
location.country_id              int64
location.locality_verbose       object
location.zipcode                object
location.locality               object
location.longitude              object
dtype: object

In [59]:
'''
Data cleaning for DF columns
'''
# Title casing
data.columns = [value.title() for value in data.columns.values]

# Replacing _ with " " 
data.columns = [value.replace("_"," ") for value in data.columns.values]

# Removing key name if present
data.columns = [value.split(".")[1] if "." in value else value for value in data.columns.values]

# Renaming Name, ID columns
data.rename(columns={
    'Name': 'Restaurant Name',
    'Id': 'Restaurant ID'
}, inplace=True)

In [60]:
# Confirm new column names
data.columns

Index(['Has Online Delivery', 'Price Range', 'Restaurant Name',
       'Has Table Booking', 'Is Delivering Now', 'Average Cost For Two',
       'Switch To Order Menu', 'Cuisines', 'Currency', 'Restaurant ID',
       'Establishment Types', 'Rating Text', 'Rating Color', 'Votes',
       'Aggregate Rating', 'Latitude', 'Address', 'City', 'City Id',
       'Country Id', 'Locality Verbose', 'Zipcode', 'Locality', 'Longitude'],
      dtype='object')

In [61]:
'''
Knowing what we know the contents of each JSON file, lets further explore each item in the list. 
Checking for the existence of a `restaurants` key, since that is where the data is stored.

Findings:
- certain results do not contain restaurant info because those calls exceeded the API limit
- add logic to filter out results that do not contain the 'restaurants' key
'''
i=0
for result in df:
    if 'restaurants' in result.keys(): 
        if len(result['restaurants']) == 0:
            i+=1
            continue
        else:
            i+=1
            continue
    else: 
        i+=1
        print(i)
        print(result['message'])
        print(result.keys())

71
API limit exceeded
dict_keys(['message', 'code', 'status'])
72
API limit exceeded
dict_keys(['message', 'code', 'status'])
73
API limit exceeded
dict_keys(['message', 'code', 'status'])
74
API limit exceeded
dict_keys(['message', 'code', 'status'])
75
API limit exceeded
dict_keys(['message', 'code', 'status'])
76
API limit exceeded
dict_keys(['message', 'code', 'status'])
77
API limit exceeded
dict_keys(['message', 'code', 'status'])
78
API limit exceeded
dict_keys(['message', 'code', 'status'])
79
API limit exceeded
dict_keys(['message', 'code', 'status'])
80
API limit exceeded
dict_keys(['message', 'code', 'status'])
81
API limit exceeded
dict_keys(['message', 'code', 'status'])
82
API limit exceeded
dict_keys(['message', 'code', 'status'])
83
API limit exceeded
dict_keys(['message', 'code', 'status'])
84
API limit exceeded
dict_keys(['message', 'code', 'status'])
85
API limit exceeded
dict_keys(['message', 'code', 'status'])
86
API limit exceeded
dict_keys(['message', 'code', 'st

In [62]:
'''
List comprehension to reduce the final list of items further, eliminating the continuous nested dict with:

`len(result['restaurants']) >0`

while also adding logic to filter for values with `restaurant` in keys
'''
restaurants = [result['restaurants'] for result in df if 'restaurants' in result.keys() and len(result['restaurants']) >0]

In [63]:
'''
Final result for processing data from a single JSON file
- Create a DF on the outside of the loop
- Add login produced above to filter out irrelevant values
- Normalize data; process headers
- Append results to outer DF
'''

final = pd.DataFrame([])

for restaurant_list in restaurants:
    
    restaurant_list = [result for result in restaurant_list if len(result['restaurant'].keys())>1]
    for restaurant in restaurant_list:
        
        restaurant = restaurant['restaurant']
        
        data = json_normalize(restaurant)
        data = data[keep]

        data.columns = [value.title() for value in data.columns.values]
        data.columns = [value.replace("_"," ") for value in data.columns.values]
        data.columns = [value.split(".")[1] if "." in value else value for value in data.columns.values]

        data.rename(
            columns={
                'Name': 'Restaurant Name',
                'Id': 'Restaurant ID'
            }
        , inplace=True)

        final = final.append(data, ignore_index=True)
        print('Addedd row - result set is now ', final.shape[0], ' rows')

Addedd row - result set is now  1  rows
Addedd row - result set is now  2  rows
Addedd row - result set is now  3  rows
Addedd row - result set is now  4  rows
Addedd row - result set is now  5  rows
Addedd row - result set is now  6  rows
Addedd row - result set is now  7  rows
Addedd row - result set is now  8  rows
Addedd row - result set is now  9  rows
Addedd row - result set is now  10  rows
Addedd row - result set is now  11  rows
Addedd row - result set is now  12  rows
Addedd row - result set is now  13  rows
Addedd row - result set is now  14  rows
Addedd row - result set is now  15  rows
Addedd row - result set is now  16  rows
Addedd row - result set is now  17  rows
Addedd row - result set is now  18  rows
Addedd row - result set is now  19  rows
Addedd row - result set is now  20  rows
Addedd row - result set is now  21  rows
Addedd row - result set is now  22  rows
Addedd row - result set is now  23  rows
Addedd row - result set is now  24  rows
Addedd row - result set i

Addedd row - result set is now  199  rows
Addedd row - result set is now  200  rows
Addedd row - result set is now  201  rows
Addedd row - result set is now  202  rows
Addedd row - result set is now  203  rows
Addedd row - result set is now  204  rows
Addedd row - result set is now  205  rows
Addedd row - result set is now  206  rows
Addedd row - result set is now  207  rows
Addedd row - result set is now  208  rows
Addedd row - result set is now  209  rows
Addedd row - result set is now  210  rows
Addedd row - result set is now  211  rows
Addedd row - result set is now  212  rows
Addedd row - result set is now  213  rows
Addedd row - result set is now  214  rows
Addedd row - result set is now  215  rows
Addedd row - result set is now  216  rows
Addedd row - result set is now  217  rows
Addedd row - result set is now  218  rows
Addedd row - result set is now  219  rows
Addedd row - result set is now  220  rows
Addedd row - result set is now  221  rows
Addedd row - result set is now  22

Addedd row - result set is now  397  rows
Addedd row - result set is now  398  rows
Addedd row - result set is now  399  rows
Addedd row - result set is now  400  rows
Addedd row - result set is now  401  rows
Addedd row - result set is now  402  rows
Addedd row - result set is now  403  rows
Addedd row - result set is now  404  rows
Addedd row - result set is now  405  rows
Addedd row - result set is now  406  rows
Addedd row - result set is now  407  rows
Addedd row - result set is now  408  rows
Addedd row - result set is now  409  rows
Addedd row - result set is now  410  rows
Addedd row - result set is now  411  rows
Addedd row - result set is now  412  rows
Addedd row - result set is now  413  rows
Addedd row - result set is now  414  rows
Addedd row - result set is now  415  rows
Addedd row - result set is now  416  rows
Addedd row - result set is now  417  rows
Addedd row - result set is now  418  rows
Addedd row - result set is now  419  rows
Addedd row - result set is now  42

In [64]:
final.head()

,Has Online Delivery,Price Range,Restaurant Name,Has Table Booking,Is Delivering Now,Average Cost For Two,Switch To Order Menu,Cuisines,Currency,Restaurant ID,...,Aggregate Rating,Latitude,Address,City,City Id,Country Id,Locality Verbose,Zipcode,Locality,Longitude
0,0,2,The Coop,0,0,25,0,"Southern, Cajun, Soul Food",$,17066603,...,3.6,28.5973660000,"610 W Morse Boulevard, Winter Park, FL 32789",Orlando,601,216,"Winter Park, Orlando",32789,Winter Park,-81.3572190000
1,0,4,Maggiano's Little Italy,0,0,50,0,Italian,$,17059541,...,4.4,28.4332350000,"9101 International Drive,Orlando, FL 32819",Orlando,601,216,"I-Drive/Universal, Orlando",32819,I-Drive/Universal,-81.4714470000
2,0,1,Tako Cheena by Pom Pom,0,0,10,0,"Asian, Latin American, Vegetarian",$,17064405,...,4.4,28.5578450000,"932 North Mills Avenue, Orlando, FL 32803",Orlando,601,216,"Mills 50, Orlando",32803,Mills 50,-81.3645470000
3,0,3,Bosphorous Turkish Cuisine,0,0,40,0,"Mediterranean, Turkish",$,17057797,...,4.2,28.5976271000,"108 S Park Ave, Winter Park, FL 32789",Orlando,601,216,"Winter Park, Orlando",32789,Winter Park,-81.3508344000
4,0,3,Bahama Breeze Island Grille,0,0,45,0,Caribbean,$,17057591,...,4.3,28.4370650000,"8849 International Drive, Orlando, FL 32819",Orlando,601,216,"I-Drive/Universal, Orlando",32819,I-Drive/Universal,-81.4715260000


In [ ]:
'''
Now, we broaden the process to encapsulate all JSON files

To do:
- Time results when using parallel computations (Dask)
'''

json_files = glob("*.json")

final = pd.DataFrame([])
for file in json_files:

    with open(file) as f:
    
        data = json.load(f)
        
        restaurants = [result['restaurants'] for result in data \
                       if 'restaurants' in result.keys() and \
                       len(result['restaurants']) >0]
 
        for restaurant_list in restaurants:

            restaurant_list = [result for result in restaurant_list if len(result['restaurant'].keys())>1]
            for restaurant in restaurant_list:

                restaurant = restaurant['restaurant']

                data = json_normalize(restaurant)
                data = data[keep]

                data.columns = [value.title() for value in data.columns.values]
                data.columns = [value.replace("_"," ") for value in data.columns.values]
                data.columns = [value.split(".")[1] if "." in value else value for value in data.columns.values]

                data.rename(
                    columns={
                        'Name': 'Restaurant Name',
                        'Id': 'Restaurant ID'
                    }
                , inplace=True)

                final = final.append(data, ignore_index=True)
                print('Addedd row - result set is now ', final.shape[0], ' rows')

Addedd row - result set is now  1  rows
Addedd row - result set is now  2  rows
Addedd row - result set is now  3  rows
Addedd row - result set is now  4  rows
Addedd row - result set is now  5  rows
Addedd row - result set is now  6  rows
Addedd row - result set is now  7  rows
Addedd row - result set is now  8  rows
Addedd row - result set is now  9  rows
Addedd row - result set is now  10  rows
Addedd row - result set is now  11  rows
Addedd row - result set is now  12  rows
Addedd row - result set is now  13  rows
Addedd row - result set is now  14  rows
Addedd row - result set is now  15  rows
Addedd row - result set is now  16  rows
Addedd row - result set is now  17  rows
Addedd row - result set is now  18  rows
Addedd row - result set is now  19  rows
Addedd row - result set is now  20  rows
Addedd row - result set is now  21  rows
Addedd row - result set is now  22  rows
Addedd row - result set is now  23  rows
Addedd row - result set is now  24  rows
Addedd row - result set i

Addedd row - result set is now  205  rows
Addedd row - result set is now  206  rows
Addedd row - result set is now  207  rows
Addedd row - result set is now  208  rows
Addedd row - result set is now  209  rows
Addedd row - result set is now  210  rows
Addedd row - result set is now  211  rows
Addedd row - result set is now  212  rows
Addedd row - result set is now  213  rows
Addedd row - result set is now  214  rows
Addedd row - result set is now  215  rows
Addedd row - result set is now  216  rows
Addedd row - result set is now  217  rows
Addedd row - result set is now  218  rows
Addedd row - result set is now  219  rows
Addedd row - result set is now  220  rows
Addedd row - result set is now  221  rows
Addedd row - result set is now  222  rows
Addedd row - result set is now  223  rows
Addedd row - result set is now  224  rows
Addedd row - result set is now  225  rows
Addedd row - result set is now  226  rows
Addedd row - result set is now  227  rows
Addedd row - result set is now  22

Addedd row - result set is now  420  rows
Addedd row - result set is now  421  rows
Addedd row - result set is now  422  rows
Addedd row - result set is now  423  rows
Addedd row - result set is now  424  rows
Addedd row - result set is now  425  rows
Addedd row - result set is now  426  rows
Addedd row - result set is now  427  rows
Addedd row - result set is now  428  rows
Addedd row - result set is now  429  rows
Addedd row - result set is now  430  rows
Addedd row - result set is now  431  rows
Addedd row - result set is now  432  rows
Addedd row - result set is now  433  rows
Addedd row - result set is now  434  rows
Addedd row - result set is now  435  rows
Addedd row - result set is now  436  rows
Addedd row - result set is now  437  rows
Addedd row - result set is now  438  rows
Addedd row - result set is now  439  rows
Addedd row - result set is now  440  rows
Addedd row - result set is now  441  rows
Addedd row - result set is now  442  rows
Addedd row - result set is now  44

Addedd row - result set is now  631  rows
Addedd row - result set is now  632  rows
Addedd row - result set is now  633  rows
Addedd row - result set is now  634  rows
Addedd row - result set is now  635  rows
Addedd row - result set is now  636  rows
Addedd row - result set is now  637  rows
Addedd row - result set is now  638  rows
Addedd row - result set is now  639  rows
Addedd row - result set is now  640  rows
Addedd row - result set is now  641  rows
Addedd row - result set is now  642  rows
Addedd row - result set is now  643  rows
Addedd row - result set is now  644  rows
Addedd row - result set is now  645  rows
Addedd row - result set is now  646  rows
Addedd row - result set is now  647  rows
Addedd row - result set is now  648  rows
Addedd row - result set is now  649  rows
Addedd row - result set is now  650  rows
Addedd row - result set is now  651  rows
Addedd row - result set is now  652  rows
Addedd row - result set is now  653  rows
Addedd row - result set is now  65

Addedd row - result set is now  846  rows
Addedd row - result set is now  847  rows
Addedd row - result set is now  848  rows
Addedd row - result set is now  849  rows
Addedd row - result set is now  850  rows
Addedd row - result set is now  851  rows
Addedd row - result set is now  852  rows
Addedd row - result set is now  853  rows
Addedd row - result set is now  854  rows
Addedd row - result set is now  855  rows
Addedd row - result set is now  856  rows
Addedd row - result set is now  857  rows
Addedd row - result set is now  858  rows
Addedd row - result set is now  859  rows
Addedd row - result set is now  860  rows
Addedd row - result set is now  861  rows
Addedd row - result set is now  862  rows
Addedd row - result set is now  863  rows
Addedd row - result set is now  864  rows
Addedd row - result set is now  865  rows
Addedd row - result set is now  866  rows
Addedd row - result set is now  867  rows
Addedd row - result set is now  868  rows
Addedd row - result set is now  86

...

Addedd row - result set is now  10299  rows
Addedd row - result set is now  10300  rows
Addedd row - result set is now  10301  rows
Addedd row - result set is now  10302  rows
Addedd row - result set is now  10303  rows
Addedd row - result set is now  10304  rows
Addedd row - result set is now  10305  rows
Addedd row - result set is now  10306  rows
Addedd row - result set is now  10307  rows
Addedd row - result set is now  10308  rows
Addedd row - result set is now  10309  rows
Addedd row - result set is now  10310  rows
Addedd row - result set is now  10311  rows
Addedd row - result set is now  10312  rows
Addedd row - result set is now  10313  rows
Addedd row - result set is now  10314  rows
Addedd row - result set is now  10315  rows
Addedd row - result set is now  10316  rows
Addedd row - result set is now  10317  rows
Addedd row - result set is now  10318  rows
Addedd row - result set is now  10319  rows
Addedd row - result set is now  10320  rows
Addedd row - result set is now  

Addedd row - result set is now  10490  rows
Addedd row - result set is now  10491  rows
Addedd row - result set is now  10492  rows
Addedd row - result set is now  10493  rows
Addedd row - result set is now  10494  rows
Addedd row - result set is now  10495  rows
Addedd row - result set is now  10496  rows
Addedd row - result set is now  10497  rows
Addedd row - result set is now  10498  rows
Addedd row - result set is now  10499  rows
Addedd row - result set is now  10500  rows
Addedd row - result set is now  10501  rows
Addedd row - result set is now  10502  rows
Addedd row - result set is now  10503  rows
Addedd row - result set is now  10504  rows
Addedd row - result set is now  10505  rows
Addedd row - result set is now  10506  rows
Addedd row - result set is now  10507  rows
Addedd row - result set is now  10508  rows
Addedd row - result set is now  10509  rows
Addedd row - result set is now  10510  rows
Addedd row - result set is now  10511  rows
Addedd row - result set is now  

Addedd row - result set is now  10677  rows
Addedd row - result set is now  10678  rows
Addedd row - result set is now  10679  rows
Addedd row - result set is now  10680  rows
Addedd row - result set is now  10681  rows
Addedd row - result set is now  10682  rows
Addedd row - result set is now  10683  rows
Addedd row - result set is now  10684  rows
Addedd row - result set is now  10685  rows
Addedd row - result set is now  10686  rows
Addedd row - result set is now  10687  rows
Addedd row - result set is now  10688  rows
Addedd row - result set is now  10689  rows
Addedd row - result set is now  10690  rows
Addedd row - result set is now  10691  rows
Addedd row - result set is now  10692  rows
Addedd row - result set is now  10693  rows
Addedd row - result set is now  10694  rows
Addedd row - result set is now  10695  rows
Addedd row - result set is now  10696  rows
Addedd row - result set is now  10697  rows
Addedd row - result set is now  10698  rows
Addedd row - result set is now  

Addedd row - result set is now  10870  rows
Addedd row - result set is now  10871  rows
Addedd row - result set is now  10872  rows
Addedd row - result set is now  10873  rows
Addedd row - result set is now  10874  rows
Addedd row - result set is now  10875  rows
Addedd row - result set is now  10876  rows
Addedd row - result set is now  10877  rows
Addedd row - result set is now  10878  rows
Addedd row - result set is now  10879  rows
Addedd row - result set is now  10880  rows
Addedd row - result set is now  10881  rows
Addedd row - result set is now  10882  rows
Addedd row - result set is now  10883  rows
Addedd row - result set is now  10884  rows
Addedd row - result set is now  10885  rows
Addedd row - result set is now  10886  rows
Addedd row - result set is now  10887  rows
Addedd row - result set is now  10888  rows
Addedd row - result set is now  10889  rows
Addedd row - result set is now  10890  rows
Addedd row - result set is now  10891  rows
Addedd row - result set is now  

Addedd row - result set is now  11062  rows
Addedd row - result set is now  11063  rows
Addedd row - result set is now  11064  rows
Addedd row - result set is now  11065  rows
Addedd row - result set is now  11066  rows
Addedd row - result set is now  11067  rows
Addedd row - result set is now  11068  rows
Addedd row - result set is now  11069  rows
Addedd row - result set is now  11070  rows
Addedd row - result set is now  11071  rows
Addedd row - result set is now  11072  rows
Addedd row - result set is now  11073  rows
Addedd row - result set is now  11074  rows
Addedd row - result set is now  11075  rows
Addedd row - result set is now  11076  rows
Addedd row - result set is now  11077  rows
Addedd row - result set is now  11078  rows
Addedd row - result set is now  11079  rows
Addedd row - result set is now  11080  rows
Addedd row - result set is now  11081  rows
Addedd row - result set is now  11082  rows
Addedd row - result set is now  11083  rows
Addedd row - result set is now  

In [ ]:
final.shape

In [ ]:
final.head()

In [ ]:
'''
Doing some exploratory analysis in the following kernels. The idea is to:
- Get a sense of what the range of values are for specific columns
- Whether there are significnat null values (can also do this w/ final.info())
- Identify values that require mapping (or can be stored elsewhere as dimension tables)
'''
print()

In [ ]:
# Can remain as is
final['Price Range'].value_counts()

In [ ]:
# Need to convert human-readable format (Yes/No)
final['Is Delivering Now'].value_counts()

In [ ]:
# There should be a mapping for this with a standardized currency convention
# Did not add yet but will explore 
final['Currency'].value_counts()

In [ ]:
# can dig into the 0 values further - i.e., why are there 2 apparent 0 values?
final['Aggregate Rating'].value_counts(dropna=False)

In [ ]:
# votes can remain as is
final['Votes'].value_counts(dropna=False)

In [ ]:
# can remain as is
final['City'].value_counts()

In [ ]:
final.columns

In [ ]:
# New column order
new_column_order = ['Restaurant ID', 'Restaurant Name', 'Country Id', 'City', 'City Id', 'Address', 'Zipcode', 
             'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines', 'Price Range', 
             'Average Cost For Two', 'Currency', 'Establishment Types', 'Has Online Delivery', 
             'Has Table Booking', 'Is Delivering Now', 'Switch To Order Menu', 'Aggregate Rating', 
            'Rating Color', 'Rating Text', 'Votes']
len(new_column_order)

In [ ]:
# Confirm length of final columns
len(final.columns)

In [ ]:
# reorder 
final = final[new_column_order]
final.head()

In [ ]:
# delete establishment types - these are all empty lists
del final['Establishment Types']
final.head()

In [ ]:
# Review left side of DF ('...' hides columns)
final.iloc[:5, :10]

In [ ]:
# Review right side of DF ('...' hides columns)
final.iloc[:5, 10:]

In [ ]:
# Map these to No/Yes
final['Has Online Delivery'].value_counts(dropna=False)

In [ ]:
# Map these to No/Yes
final['Has Table Booking'].value_counts(dropna=False)

In [ ]:
# Map these to No/Yes
final['Is Delivering Now'].value_counts(dropna=False)

In [ ]:
# This column can be deleted
final['Switch To Order Menu'].value_counts(dropna=False)

In [ ]:
# List of boolean columns (note - `switch to order menu` can be removed (deleted further down))
boolean_columns = ['Has Online Delivery', 'Has Table Booking', 'Is Delivering Now', 'Switch To Order Menu']

# function to map values
def map_values(x):
    if x == 0:
        return 'No'
    if x == 1:
        return 'Yes'

# reassign values via applymap
final.loc[:, boolean_columns] = final.loc[:, boolean_columns].applymap(lambda x: map_values(x))

In [ ]:
# check results @ high level
final.head()

In [ ]:
'''
Rating Colors are stored as hex codes
- This is not readable, so using an API to extract color names and mapping those to the DF

Using `thecolorapi` to get values
https://www.thecolorapi.com/docs

Note - I did not cross check these hex codes, though for the purpose
of this exercise, it seems fine to use these results.

Steps:
- Extract color name for single hex code
- Package into function
- Save results for each hex code via dict comprehension


'''
# Review distribution of `Rating Color`
final['Rating Color'].value_counts(dropna=False)

In [ ]:
# Assign unique hex codes to variable
hex_codes = final['Rating Color'].unique().tolist()

In [ ]:
import requests

# params for first API call
params = {
    'hex': hex_codes[0]
}
# return application/json
headers = {
    'Content-Type': 'application/json'
}

r = requests.get(url, params=params, headers=headers)

In [ ]:
# confirm response = OK
r

In [ ]:
# load text data into python dict
json_r = json.loads(r.text)

In [ ]:
# review keys
json_r.keys()

In [ ]:
# color name is the value for the `value` key
json_r['name']

In [ ]:
# assing color name to variable
color_name = json_r['name']['value']

In [ ]:
# Packaged the above code into a function
def get_color_name(hex_code):
    url = "http://thecolorapi.com/id"
    params = {
        'hex': hex_code
    }

    headers = {
        'Content-Type': 'application/json'
    }
    
    r = requests.get(url, params=params, headers=headers)
    if r.status_code == 200:
        json_r = json.loads(r.text)
        color_name = json_r['name']['value']
        return color_name
    return r.status_code

In [ ]:
# Stored results into a dictionary using a dict comprehension
color_names = {str(hex_code): str(get_color_name(hex_code)) for hex_code in hex_codes}

In [ ]:
# review results
color_names

In [ ]:
# Map values to the DF
final.loc[:, 'Rating Color'] = final.loc[:, 'Rating Color'].map(color_names)

In [ ]:
# Confirm new values
final['Rating Color'].value_counts(dropna=False)

In [ ]:
'''
Doing additional data munging below.
- Some columns do not provide any insight (`Switch To Order Menu`) or have many Nan values (`Zipcode`)
- Other columns can be stored in a separate DF to serve as mapping files (address related data)
'''
print()

In [ ]:
# After reviewing `Zipcode` values, this field can be dropped
final['Zipcode'].value_counts(dropna=False)

In [ ]:
# removing Zipcode
del final['Zipcode']

In [ ]:
# All values are `No`, so this column can be deleted
final['Switch To Order Menu'].value_counts(dropna=False)

In [ ]:
del final['Switch To Order Menu']

In [ ]:
final['Rating Text'].value_counts(dropna=False)

In [ ]:
final['Rating Color'].value_counts(dropna=False)

In [ ]:
'''
To get country names, we can use the Zomato /ciites endpoint
- Only need to make a single API call using the city_ids
- Similar process as hex codes

--> Need to get a single city id for each country, then pass those in as a param
'''
final['Country Id'].value_counts(dropna=False)

In [ ]:
# API key for zomato (masking for upload to GitHub)
api_key = 'YOUR_API_KEY_HERE'

In [ ]:
# Get list of unique country ids
country_ids = final['Country Id'].unique().tolist()

In [ ]:
# Snipped used to validate logic in below kernel
final[final['Country Id'] == 166][['City Id']].head(1).values[0][0]

In [ ]:
# Initialize empty list to store city ids
city_ids = []

for country_id in country_ids:
    # Filters `final` to rows where `Country Id` equals `country_id`
    # Assigns first result to `city_id`
    city_id = final[final['Country Id'] == country_id][['City Id']].head(1).values[0][0]
    # Append result to outer list
    city_ids.append(city_id)

In [ ]:
# confirm lengths match
len(city_ids) == len(country_ids)

In [ ]:
# Need to wrap the list in a string object and remove the `[` and `]` characters
# Otherwise the city_ids are not picked up as a proper parameter
city_ids = str(city_ids).replace("[","").replace("]","")
city_ids

In [ ]:
# URL for the API endpoint
url = "https://developers.zomato.com/api/v2.1/cities?"

# Headers with API key and content type
headers = {
    'user-key': api_key,
    'Content-Type': 'application/json'
}
# city ids as parameter
params = {
    'city_ids': city_ids
}

In [ ]:
# Send request
r = requests.get(url, params=params, headers=headers)

In [ ]:
# req is OK
r.status_code

In [ ]:
# Convert to dict; check keys
r_json = json.loads(r.text)
r_json.keys()

In [ ]:
# Check length of `location_suggestions` 
len(r_json['location_suggestions'])

In [ ]:
# Inspect first item
r_json['location_suggestions'][0]

In [ ]:
# Country mapping produced via dict comprehension
country_mapping = {
    location['country_id']: location['country_name'] for location in r_json['location_suggestions']
}

In [ ]:
# review mapping
country_mapping

In [ ]:
# Normalize, transpose, and reset index on country mapping
country_mapping = json_normalize(country_mapping).transpose().reset_index()

In [ ]:
# rename
country_mapping.rename(columns={
    'index': 'Country Id',
    0: 'Country Name'
}, inplace=True)

In [ ]:
# sort, display results (can change index if desired)
country_mapping.sort_values('Country Id', ascending=True, inplace=True)
country_mapping

In [ ]:
# HLO of data
final.info()

In [ ]:
# confirm dtypes for Country Id
country_mapping.dtypes

In [ ]:
# confirm dtypes for Country Id match
final.dtypes

In [ ]:
# Map country name
final['Country Name'] = final['Country Id'].map(country_mapping.set_index('Country Id')['Country Name'])

In [ ]:
final.head()

In [ ]:
# Final column order
final_column_order = ['Restaurant ID', 'Restaurant Name', 'Country Name', 'City', 'Address', 
                     'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines', 'Price Range', 
                     'Average Cost For Two', 'Currency', 'Has Online Delivery', 'Has Table Booking', 
                    'Is Delivering Now', 'Aggregate Rating', 'Rating Color', 'Rating Text', 'Votes']

In [ ]:
# Reorder cols
final = final[final_column_order]

In [ ]:
final.head()

In [ ]:
# Get day, month, year values
today = datetime.now()
day = str(today.day).zfill(2)
month = str(today.month).zfill(2)
year = str(today.year)

# final filename
filename = f'Restaurant_Data_{day}-{month}-{year}.xlsx'

# Save file to excel
with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    final.to_excel(writer, sheet_name='raw_data')